In [ ]:
import sys
sys.path.append('..')
import numpy as np, elastic_rods, linkage_vis
from bending_validation import suppress_stdout as so

# Linkage representation

In [ ]:
l = elastic_rods.RodLinkage('../../examples/single_linkage.obj')
l.setMaterial(elastic_rods.RodMaterial('rectangle', 200, 0.3, np.array([9, 6]) / 1000))
driver = 0

In [ ]:
viewer = linkage_vis.LinkageViewer(l)
viewer.show()

In [ ]:
mkdir single_joint_animation

In [ ]:
from open_linkage import open_linkage
def equilibriumSolver(tgtAngle, l, opts, fv):
    opts.beta = 1e-8
    opts.gradTol = 1e-4
    opts.useIdentityMetric = False
    return elastic_rods.compute_equilibrium(l, tgtAngle, options=opts, fixedVars=fv)
numFrames = 90
amplitude = 25
for i, t in enumerate(np.linspace(0, 2 * np.pi, numFrames)):
    tgtAngle = 50 + amplitude * np.sin(t)
    with so(): open_linkage(l, driver, np.deg2rad(tgtAngle) - l.averageJointAngle, 1, viewer, zPerturbationEpsilon=0, equilibriumSolver=equilibriumSolver, maxNewtonIterationsIntermediate=20, verbose=10, useTargetAngleConstraint=True);
    l.saveVisualizationGeometry(f'single_joint_animation/frame_{i}.msh', averagedMaterialFrames=True)

# Stretch/bend/twist animations

In [ ]:
from matplotlib import pyplot as plt
# Fourier-transformed square wave with a hold at the second peak.
x = np.linspace(0, 4.0 / 3.0, 1000)
def SMaxZero(x, k): return np.log(np.exp(k * x) + 1) / k
def f(x): return SMaxZero((3 / np.pi) * (np.sin(2 * np.pi * x) + np.sin(2 * np.pi * 3 * x) / 3.0), 5)
def g(x): return f(3/4 * x - 1/8)
def perturbationMagnitude(x):
    return np.select([x < 2.0 / 3.0, x < 1.0, x >= 1.0], [g(x), g(2.0 / 3.0), g(x - 1.0 / 3.0)])
    
plt.plot(x, perturbationMagnitude(x))
plt.show()

In [ ]:
import time
numFrames = 120

rodLength = 15
nsubdiv = 120
npts = nsubdiv + 1
pts = np.linspace([-rodLength / 2, 0, 0], [rodLength / 2, 0, 0], nsubdiv)
animated_rod = elastic_rods.ElasticRod(pts)
animated_rod.setMaterial(elastic_rods.RodMaterial('rectangle', 200, 0.3, [1, 0.5]))
origPts = np.array(animated_rod.deformedPoints())
origThetas = animated_rod.thetas()

In [ ]:
preview = linkage_vis.LinkageViewer(animated_rod)
preview.show()

In [ ]:
eopts = elastic_rods.NewtonOptimizerOptions()
eopts.niter = 1000

In [ ]:
def stretchRod(r, magnitude):
    stretchedPts = origPts.copy()
    stretchedPts[:, 0] *= 1 + 0.2 * magnitude
    r.setDeformedConfiguration(stretchedPts, origThetas)
def bendRod(r, magnitude):
    nv = r.numVertices()
    midPt = nv // 2
    bentPoints = origPts.copy()
    disp = 3 * magnitude
    bentPoints[0, 1] = -disp
    bentPoints[midPt, 1] = disp
    bentPoints[nv - 1, 1] = -disp
    r.setDeformedConfiguration(bentPoints, origThetas)
    fixedVars = [1, 3 * midPt + 1, 3 * (nv - 1) + 1]
    fixedVars += [3 * midPt, 3 * nv] # pin rigid motion
    with so(): elastic_rods.compute_equilibrium(r, eopts, fixedVars)
def twistRod(r, magnitude):
    nv = r.numVertices()
    twistedThetas = np.array(r.thetas()).copy()
    twistedThetas[0]  = -magnitude * np.pi
    twistedThetas[-1] =  magnitude * np.pi
    r.setDeformedConfiguration(origPts, twistedThetas)
    fixedVars = [3 * nv, r.numDoF() - 1]
    # We actually need to fix the two end edges entirely to prevent
    # the rod from exploiting parallel transport to untwist its ends!
    fixedVars += [0, 1, 2, 3, 4, 5, 3 * (nv - 2), 3 * (nv - 2) + 1, 3 * (nv - 2) + 2, 3 * (nv - 1), 3 * (nv - 1) + 1, 3 * (nv - 1) + 2] # pin rigid motion
    with so(): elastic_rods.compute_equilibrium(r, eopts, fixedVars)
    
def runAnimation(name, deformer):
    for frame, t in enumerate(np.linspace(0, 4.0 / 3.0, numFrames)):
        deformer(animated_rod, perturbationMagnitude(t))
        preview.update(False)
        time.sleep(0.05)
        animated_rod.saveVisualizationGeometry(f'{name}_animation/{frame}.obj', averagedMaterialFrames=True)
#runAnimation('stretch', stretchRod)
#runAnimation('bend', bendRod)
runAnimation('twist', twistRod)

# Cross-section comparisons

In [ ]:
rectMat = elastic_rods.RodMaterial('rectangle', 200, 0.3, [0.56360235, 0.56360235])
plusMat = elastic_rods.RodMaterial('+', 200, 0.3, [1, 1, 0.1, 0.1])

print(rectMat.bendingStiffness.lambda_1,  rectMat.bendingStiffness.lambda_2)
print(plusMat.bendingStiffness.lambda_1,  plusMat.bendingStiffness.lambda_2)
print(rectMat.twistingStiffness, plusMat.twistingStiffness)

In [ ]:
rectMat = elastic_rods.RodMaterial('rectangle', 200, 0.3, [0.33701815, 0.33701815])
plusMat = elastic_rods.RodMaterial('+', 200, 0.3, [0.5, 0.5, 0.1, 0.1])

print(rectMat.bendingStiffness.lambda_1,  rectMat.bendingStiffness.lambda_2)
print(plusMat.bendingStiffness.lambda_1,  plusMat.bendingStiffness.lambda_2)
print(rectMat.twistingStiffness, plusMat.twistingStiffness)

In [ ]:
import time
rodLength = 5
nsubdiv = 120
npts = nsubdiv + 1

pts = np.linspace([-rodLength / 2, 0, 0], [rodLength / 2, 0, 0], nsubdiv)
animated_rod = elastic_rods.ElasticRod(pts)
origPts = np.array(animated_rod.deformedPoints())
origThetas = animated_rod.thetas()

animated_rod.setMaterial(plusMat)

In [ ]:
preview = linkage_vis.LinkageViewer(animated_rod)
preview.show()

In [ ]:
eopts = elastic_rods.NewtonOptimizerOptions()

In [ ]:
from numpy.linalg import norm
numFrames = 60
def bendRod(r, magnitude):
    nv = r.numVertices()
    midPt = nv // 2
    bentPoints = np.array(r.deformedPoints())
    disp = magnitude
    bentPoints[0, 1] = -disp
    bentPoints[midPt, 1] = disp
    bentPoints[nv - 1, 1] = -disp
    r.setDeformedConfiguration(bentPoints, origThetas)
    fixedVars = [1, 3 * midPt + 1, 3 * (nv - 1) + 1]
    fixedVars += [3 * midPt, 3 * nv] # pin rigid motion
    with so(): elastic_rods.compute_equilibrium(r, eopts, fixedVars)
    return norm(r.gradient())

def twistRod(r, magnitude):
    nv = r.numVertices()
    midPt = nv // 2
    twistedThetas = np.array(r.thetas())
    twistedThetas[0]  = -magnitude
    twistedThetas[-1] =  magnitude
    r.setDeformedConfiguration(origPts, twistedThetas)
    fixedVars = [3 * nv, r.numDoF() - 1]
    fixedVars += [0, 1, 2, 3 * (nv - 1), 3 * (nv - 1) + 1, 3 * (nv - 1) + 2] # pin rigid motion
    with so(): elastic_rods.compute_equilibrium(r, eopts, fixedVars)
    return norm(r.gradient())
    
def runAnimation(name, deformer, amplitude):
    forces = []
    for frame, t in enumerate(np.linspace(0, 2 * np.pi, numFrames)):
        forces.append(deformer(animated_rod, amplitude * np.sin(t)))
        preview.update(False)
        time.sleep(0.016)
        animated_rod.saveVisualizationGeometry(f'{name}/frame_{frame}.msh', averagedMaterialFrames=True)
    return forces
animated_rod.setMaterial(plusMat)
twistMag = 0.5 * np.pi
plus_bend_forces  = runAnimation('plus_bend', bendRod, 0.5)
plus_twist_forces = runAnimation('plus_twist', twistRod, twistMag)
animated_rod.setMaterial(rectMat)
rect_bend_forces  = runAnimation('square_bend', bendRod, 0.5)
rect_twist_forces = runAnimation('square_twist', twistRod, (plusMat.twistingStiffness / rectMat.twistingStiffness) * twistMag)

In [ ]:
from matplotlib import pyplot as plt
plt.plot(plus_bend_forces)
plt.plot(rect_bend_forces)

In [ ]:
from matplotlib import pyplot as plt
plt.plot(plus_twist_forces)
plt.plot(rect_twist_forces)